## Import libraries

In [ ]:
from google.colab import drive
from pathlib import Path
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import time
import os
import csv
import concurrent.futures

## Utility functions

### Create annot and load descriptors

In [ ]:
def create_annot(path):
  image_list = list(Path(path).glob('*/*.jpg'))
  # the identity name is in the path (the name of the parent directory)
  names_list = [i.parent.name for i in image_list]  # get the identity of each image
  # keep info in a pandas DataFrame
  annot = pd.DataFrame({'identity': names_list, 'image_path': image_list})
  return annot

def concatenate_annots(list_of_paths):
  concat_annot = pd.DataFrame()
  with concurrent.futures.ThreadPoolExecutor() as executor:
    annots = [executor.submit(create_annot, path) for path in list_of_paths]
    for annot in annots:
      new_annot = annot.result()
      concat_annot = concat_annot.append(new_annot, ignore_index = True)
    return concat_annot

In [ ]:
def load_descriptors(path):
  with open(path, 'rb') as file:
    return np.load(file)

def concatenate_descriptors(list_of_paths):
  concat_descriptors = None
  with concurrent.futures.ThreadPoolExecutor() as executor:
    descriptors = [executor.submit(load_descriptors, path) for path in list_of_paths]
    for descriptor in descriptors:
      new_descriptor = descriptor.result()
      if concat_descriptors is None:
        concat_descriptors = new_descriptor
      else:
        concat_descriptors = np.concatenate([concat_descriptors, new_descriptor])
    return concat_descriptors

### Create pivots

In [ ]:
def generate_pivots(descriptors, n, strategy="rnd"):
  if strategy == "kMED":
    kmedoids = sklearn_extra.cluster.KMedoids(n_clusters=n).fit(descriptors)
    return kmedoids.cluster_centers_
  if strategy != "rnd":
    print(strategy, "was not implemented. Random pivots were returned")
  pivots_id = np.random.choice(np.arange(len(descriptors)), size=n)
  return descriptors[pivots_id]

def generate_list_of_pivots(descriptors, t, n, strategy="rnd"):
  list_of_pivots = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    pivots = [executor.submit(generate_pivots, descriptors, n, strategy) for i in range(t)]
    for pivot in concurrent.futures.as_completed(pivots):
      new_pivot = pivot.result()
      list_of_pivots.append(new_pivot)
    return list_of_pivots

### Save test results

In [ ]:
def save_results(dir, file_name, results):
  with open(os.path.join(dir, file_name +".csv"), 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(["CLASS", "AP", "QUERY TIME"])
    # write the data
    for r in results:
      writer.writerow(r) 

## Test Performance

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Create annot and load descriptors for the database

In [ ]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/AutoEncoder/512to128withPace64.npy','/content/drive/MyDrive/CV_Birds/features/distractor/AutoEncoder/512to128withPace64.npy'])
db_descriptors.shape

(72332, 128)

### Create annot and load descriptors for the test set

In [ ]:
query_annot = create_annot('/content/drive/MyDrive/CV_Birds/test')
query_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
...,...,...
1620,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1621,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1622,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1623,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...


In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/AutoEncoder/512to128withPace64.npy')
query_descriptors.shape

(1625, 128)

To run our tests we select only the first image of each species within the test set. Please note that within the test set we have 5 images per species.

In [ ]:
queries_indexes = [x for x in range(325*5) if x%5 == 0]

### Create PP-Index

In [ ]:
def get_descriptor_from_id(id_object):
  return db_descriptors[id_object]

In [ ]:
%cd "/content/drive/MyDrive/CV_Birds/Notebooks/PP-Index"
%run PPIndex.ipynb

/content/drive/.shortcut-targets-by-id/1rI5YNBuaSlCB__w522WEkHjw-nFuvIo0/CV_Birds/Notebooks/PP-Index


In [ ]:
# generate pivots
list_of_pivots = generate_list_of_pivots(db_descriptors, t=3, n=40, strategy="rnd")

In [ ]:
list_of_pivots

[array([[ 7.002038 ,  0.       ,  7.4229164, ...,  7.455193 ,  2.572634 ,
          4.384415 ],
        [10.483301 ,  0.       ,  6.2527003, ..., 13.816943 , 13.065234 ,
          7.481996 ],
        [ 7.352996 ,  0.       ,  8.766191 , ...,  3.4748816, 10.845868 ,
          3.3058126],
        ...,
        [ 9.526421 ,  0.       ,  6.1654005, ...,  8.1919985,  9.146018 ,
          9.760849 ],
        [15.325872 ,  0.       ,  2.7922618, ...,  7.9675   ,  9.636958 ,
          4.7283134],
        [ 9.64761  ,  0.       ,  8.028699 , ...,  2.2090583, 15.94067  ,
          4.6010737]], dtype=float32),
 array([[13.2457905,  0.       ,  7.0893197, ..., 10.441069 , 11.8520775,
          9.513583 ],
        [ 5.295755 ,  0.       , 11.533067 , ...,  7.240908 ,  9.353029 ,
          3.292447 ],
        [ 8.760872 ,  0.       , 18.811417 , ..., 11.877479 , 12.0398035,
          9.048934 ],
        ...,
        [ 5.2419567,  0.       ,  6.6620975, ...,  7.5224104,  6.520434 ,
          5.6059065

In [ ]:
# cosine forest
cosine_forest = PrefixForest(list_of_pivots, length=3, distance_metric='cosine', base_directory="/content/cosine", forest_file='forest_structure')

[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Forest is still empty


In [ ]:
cosine_forest.insert_objects_into_forest(range(len(db_descriptors)))
cosine_forest.save()

Inserting objects into tree1 ...
Inserting objects into tree2 ...
Inserting objects into tree3 ...
--- Insertion into tree tree2 completed: 250.331 seconds ---
--- Insertion into tree tree3 completed: 250.453 seconds ---
--- Insertion into tree tree1 completed: 250.959 seconds ---


In [ ]:
!cp /content/cosine/tree1* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/cosine/
!cp /content/cosine/tree2* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/cosine/
!cp /content/cosine/tree3* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/cosine/
!cp /content/cosine/forest* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/cosine/

In [ ]:
# euclidean forest
euclidean_forest = PrefixForest(list_of_pivots, length=3, distance_metric='euclidean', base_directory="/content/euclidean", forest_file='forest_structure')

[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Forest is still empty


In [ ]:
euclidean_forest.insert_objects_into_forest(range(len(db_descriptors)))
euclidean_forest.save()

Inserting objects into tree1 ...
Inserting objects into tree2Inserting objects into ... tree3 ...

--- Insertion into tree tree2 completed: 220.925 seconds ---
--- Insertion into tree tree1 completed: 221.721 seconds ---
--- Insertion into tree tree3 completed: 221.903 seconds ---


In [ ]:
!cp /content/euclidean/tree1* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/euclidean/
!cp /content/euclidean/tree2* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/euclidean/
!cp /content/euclidean/tree3* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/euclidean/
!cp /content/euclidean/forest* /content/drive/MyDrive/CV_Birds/indexes/fine_tuning/forest/euclidean/

### Compute mAP

In [ ]:
birds_db = db_annot.loc[db_annot['identity'] != 'mirflickr']
counts = birds_db.groupby('identity').count()
print("Minimum number of images per species:", int(counts.min()))
print("Maximum number of images per species:", int(counts.max()))
print("Average number of images:", float(counts.sum()/325))

Minimum number of images per species: 116
Maximum number of images per species: 249
Average number of images: 145.63692307692307


Since at most we have 249 images per species, we use $n=250$.

In [ ]:
n = 250

The formula for Average Precision is the following:

> $AP@n=\frac{1}{GTP}\sum_{k=1}^{n}P@k×rel@k$

where $GTP$ refers to the total number of ground truth positives, $n$ refers to the total number of images we are interested in, $P@k$ refers to the precision@k and $rel@k$ is a relevance function. 

The relevance function is an indicator function which equals 1 if the document at rank $k$ is relevant and equals to 0 otherwise.

In [ ]:
def compute_ap(query_index, retrieved_ids):
  query_identity = query_annot['identity'][query_index]
  print(query_index//5, query_identity)
  GTP = len(db_annot.loc[db_annot['identity'] == query_identity])
  relevant = 0
  precision_summation = 0
  for k, id in enumerate(retrieved_ids):
    if db_annot['identity'][id] == query_identity: # relevant result
      relevant = relevant + 1
      precision_at_k = relevant/(k+1)
      precision_summation = precision_summation + precision_at_k
  return (query_identity, precision_summation/GTP)

For each query, $Q$, we can calculate a corresponding $AP$. Then, the $mAP$ is simply the mean of all the queries that were made.
> $mAP = \frac{1}{N}\sum_{i=1}^{N}AP_i$

In our case, $N=325$ (one query per species)

#### Simple forest

##### Cosine

In [ ]:
def cosine_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_forest.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.9755838521416786, 1.7659733295440674),
 ('AFRICAN FIREFINCH', 0.6589158435462069, 1.686537504196167),
 ('ALBATROSS', 0.8229655528430524, 1.5645015239715576),
 ('ALEXANDRINE PARAKEET', 0.6939855896933901, 1.5597269535064697),
 ('AMERICAN AVOCET', 0.8537752954438933, 1.4775397777557373),
 ('AMERICAN BITTERN', 0.3306831889253228, 1.5846881866455078),
 ('AMERICAN COOT', 0.978941767473708, 1.6030323505401611),
 ('AMERICAN GOLDFINCH', 0.7934199644873016, 1.5812790393829346),
 ('AMERICAN KESTREL', 0.514332494788925, 1.555809497833252),
 ('AMERICAN PIPIT', 0.9730455327684322, 1.4736847877502441),
 ('AMERICAN REDSTART', 0.20291006376856255, 1.6373004913330078),
 ('ANHINGA', 0.5906092930501899, 1.703960657119751),
 ('ANNAS HUMMINGBIRD', 0.4798049732654127, 1.7707092761993408),
 ('ANTBIRD', 0.18857007749509025, 1.8763618469238281),
 ('ARARIPE MANAKIN', 0.41522323685172285, 1.7765870094299316),
 ('ASIAN CRESTED IBIS', 0.42377686802173975, 1.663700819015503),
 ('BALD EA

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.5921329666206043
avg. query time:  1.63974518262423


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_forest_cosine_results', aps)

##### Euclidean

In [ ]:
def euclidean_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_forest.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.27635997481623614, 1.5425512790679932),
 ('AFRICAN FIREFINCH', 0.38072375436540273, 1.5756831169128418),
 ('ALBATROSS', 0.5902927728594994, 1.534177303314209),
 ('ALEXANDRINE PARAKEET', 0.6494671826721017, 1.6287510395050049),
 ('AMERICAN AVOCET', 0.5901411120194624, 1.6188430786132812),
 ('AMERICAN BITTERN', 0.6931245473093764, 1.5177760124206543),
 ('AMERICAN COOT', 0.8661172953884694, 1.5679314136505127),
 ('AMERICAN GOLDFINCH', 0.7165048007536974, 1.667614221572876),
 ('AMERICAN KESTREL', 0.12425583475897865, 1.5407147407531738),
 ('AMERICAN PIPIT', 0.8838002652240209, 1.5736517906188965),
 ('AMERICAN REDSTART', 0.08791029054550306, 1.5915522575378418),
 ('ANHINGA', 0.4902285722144563, 1.624765157699585),
 ('ANNAS HUMMINGBIRD', 0.498326472435664, 1.5923802852630615),
 ('ANTBIRD', 0.07083844170649078, 1.623838186264038),
 ('ARARIPE MANAKIN', 0.3330811433428686, 1.5737273693084717),
 ('ASIAN CRESTED IBIS', 0.26429414156587694, 1.5184755325317383),
 ('BALD

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.4477124323342157
avg. query time:  1.6049307918548583


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_forest_euclidean_results', aps)

#### Forest with query perturbation

##### Cosine

In [ ]:
def cosine_pert_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_forest.find_nearest_neighbors(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_pert_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.9737354045602129, 3.7562248706817627),
 ('AFRICAN FIREFINCH', 0.6982930883396028, 3.4007210731506348),
 ('ALBATROSS', 0.8384640409909178, 3.378161668777466),
 ('ALEXANDRINE PARAKEET', 0.8080858678039774, 3.585010290145874),
 ('AMERICAN AVOCET', 0.8923019821615076, 3.149580955505371),
 ('AMERICAN BITTERN', 0.7831969934988081, 3.3059942722320557),
 ('AMERICAN COOT', 0.978194652101179, 3.2794346809387207),
 ('AMERICAN GOLDFINCH', 0.7933392126977131, 3.518949270248413),
 ('AMERICAN KESTREL', 0.7050153382624884, 3.440474033355713),
 ('AMERICAN PIPIT', 0.9937819625957217, 3.3841099739074707),
 ('AMERICAN REDSTART', 0.586771321243714, 3.397857904434204),
 ('ANHINGA', 0.5945849706466902, 3.537003517150879),
 ('ANNAS HUMMINGBIRD', 0.7425219207188151, 3.447955369949341),
 ('ANTBIRD', 0.18216465047518055, 3.5623085498809814),
 ('ARARIPE MANAKIN', 0.7798957348678308, 3.536445140838623),
 ('ASIAN CRESTED IBIS', 0.41792687981454324, 3.349618911743164),
 ('BALD EAGLE', 0.

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.6774144236836455
avg. query time:  3.431286701789269


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_pert_forest_cosine_results', aps)

##### Euclidean

In [ ]:
def euclidean_pert_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_forest.find_nearest_neighbors(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_pert_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.30006959341569267, 3.296574831008911),
 ('AFRICAN FIREFINCH', 0.44150916606453877, 3.7050654888153076),
 ('ALBATROSS', 0.6987267977411122, 3.360330820083618),
 ('ALEXANDRINE PARAKEET', 0.7600189478915786, 3.3211324214935303),
 ('AMERICAN AVOCET', 0.7403647330722543, 3.2635700702667236),
 ('AMERICAN BITTERN', 0.7729605930728851, 3.331660747528076),
 ('AMERICAN COOT', 0.8478220942012704, 3.3243935108184814),
 ('AMERICAN GOLDFINCH', 0.7176446706973181, 3.5894525051116943),
 ('AMERICAN KESTREL', 0.15765932031848748, 3.322195053100586),
 ('AMERICAN PIPIT', 0.8881650114409287, 3.4709601402282715),
 ('AMERICAN REDSTART', 0.1277439602474703, 3.2301721572875977),
 ('ANHINGA', 0.503286727651943, 3.4237561225891113),
 ('ANNAS HUMMINGBIRD', 0.509714714576717, 3.2686939239501953),
 ('ANTBIRD', 0.08826998386008036, 3.268172264099121),
 ('ARARIPE MANAKIN', 0.43432495539741267, 3.2959392070770264),
 ('ASIAN CRESTED IBIS', 0.27439570158939697, 3.049508571624756),
 ('BALD EA

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.5210051334636614
avg. query time:  3.389532574873704


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_pert_forest_euclidean_results', aps)